In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
from sqlite3 import Error
import os

try:
    con = sqlite3.connect('DBproject.db')
except sqlite3.Error as e:
    print("SQLite 연결 오류:", e)

database_file = 'DBproject.db'
output_file = '머슬팩토리.xlsx'

if os.path.exists(database_file):
    conn = sqlite3.connect(database_file)
    table_exists = True
else:
    conn = sqlite3.connect(database_file)
    table_exists = False

if not table_exists:    
    conn.execute('''CREATE TABLE Member ( 
    Member_id text NOT NULL,
    Name text Not NULL,
    Birthday text Not NULL,
    Age_group text Not NULL,
    Address text Not NULL,
    Phone text Not NULL,
    Gender text Not NULL,
    Locker_useage Not NULL,
    Locker_id int NULL,
    Trainer_id text NULL,
    Administer_id int NOT NULL,
    TotalCost int Not NULL,
    
    primary key (Member_id),
    foreign key (Age_group) references Cost(Age_group),
    foreign key (Locker_id) references Locker(Locker_id),
    foreign key (Trainer_id) references Trainer(Trainer_id),
    foreign key (Administer_id) references Administer(Administer_id)
    );''')
    
    conn.execute('''CREATE TABLE Administer ( 
    Administer_id int NOT NULL,
    Administer_name text NOT NULL,
    
    primary key (Administer_id)
    );''')
    
    conn.execute('''CREATE TABLE Locker ( 
    Locker_id int NOT NULL,
    Locker_gender text NOT NULL,
    Administer_id int NOT NULL,
    
    primary key (Locker_id)
    foreign key (Administer_id) references Administer(Administer_id)
    );''')
    
    conn.execute('''CREATE TABLE Trainer ( 
    Trainer_id text NOT NULL,
    Trainer_name text Not NULL,
    Administer_id int NOT NULL,

    primary key (Trainer_id),
    foreign key (Administer_id) references Administer(Administer_id) 
    );''')
    
    conn.execute('''CREATE TABLE Equipment ( 
    Equipment_id text NOT NULL,
    Equipment_Check text Not NULL,
    Administer_id int NOT NULL,
    
    primary key (Equipment_id),
    foreign key (Administer_id) references Administer(Administer_id)
    );''')
    
    conn.execute('''CREATE TABLE PT ( 
    Age_group text NULL,
    Ten int NULL,
    Twenty int NULL,
    Thirty int NULL
    );''')
    
    conn.execute('''CREATE TABLE Cost ( 
    Age_group text NULL,
    OneMonth int NULL,
    ThreeMonth int NULL, 
    SixMonth int NULL,
    Year int NULL,
    Locker_cost int NULL
    );''')
    
query = "SELECT * FROM Member;"
df_member = pd.read_sql_query(query, conn)
    
query = "SELECT * FROM Administer;"
df_administer = pd.read_sql_query(query, conn)

query = "SELECT * FROM Locker;"
df_locker = pd.read_sql_query(query, conn)

query = "SELECT * FROM Trainer;"
df_trainer = pd.read_sql_query(query, conn)

query = "SELECT * FROM Equipment;"
df_equipment = pd.read_sql_query(query, conn)

query = "SELECT * FROM PT;"
df_pt = pd.read_sql_query(query, conn)

query = "SELECT * FROM Cost;"
df_cost = pd.read_sql_query(query, conn)

with pd.ExcelWriter(output_file) as writer:
    df_locker.to_excel(writer, sheet_name='Locker', index=False)
    df_administer.to_excel(writer, sheet_name='Administer', index=False)
    df_trainer.to_excel(writer, sheet_name='Trainer', index=False)
    df_equipment.to_excel(writer, sheet_name='Equipment', index=False)
    df_pt.to_excel(writer, sheet_name='PT', index=False)
    df_cost.to_excel(writer, sheet_name='Cost', index=False)

print("Excel 파일이 생성되었습니다.")

num = 0

while (num != 4):
    print("\n\n")
    print("**************** 헬스장 관리 데이터베이스 *****************")
    print("*    1. 회원등록")                                      
    print("*    2. 트레이너 모드")                                 
    print("*    3. 관리자 모드")
    print("*    4. 종료")
    print("*********************************************************")
    
    num = int(input("번호를 선택하세요 : "))
    
    
    if(num == 1):
        
        print("\n------------ 회원등록을 시작하겠습니다 ------------")
        Member_id = input("회원ID : ")
        Name = input("이름 : ")
        Birthday = input("생년월일 : ")
        Age_group = input("연령층 : ")
        Address = input("거주지 : ")
        Phone = input("휴대폰 번호 : ")
        Gender = input("성별(남/여) : ")
        
        print("\n< 등록금 >")
        query = 'SELECT DISTINCT Age_Group, OneMonth, ThreeMonth, SixMonth, Year FROM Cost'
        cost = pd.read_sql_query(query, con)
        print(cost.to_string(index=False, col_space=15))
        
        while(1) :
            cost = int(input("원하시는 기간을 정하여 표를 보고 가격을 입력해주세요 : "))
            if (cost in [50000 ,120000, 190000, 330000, 80000, 150000, 250000, 450000, 70000, 130000, 210000, 370000]):
                break
            else :
                print("없는 가격입니다. 다시 입력해주세요.")
                
        Locker_useage = input("\n라커를 이용하시겠습니까? \n가격 - 10000원 (yes/no) : ")
        if Locker_useage == "yes" or Locker_useage == "네":
            locker_cost = 10000
        else :
            locker_cost = 0
        
        print("\n---------------- 트레이너 배정 -----------------")
        query = 'SELECT DISTINCT Trainer_id as 트레이너ID, Trainer_name as 이름 FROM Trainer'
        traniner = pd.read_sql_query(query, con)
        print(traniner.to_string(index=False, col_space=10))
        Trainer_id = input("원하시는 트레이너ID를 입력해주세요 : ")
        
        PT = input("\nPT를 하시겠습니까? (yes/no) : ")
        if (PT == "yes" or PT == "네") :
            query = 'SELECT DISTINCT * FROM PT'
            pt = pd.read_sql_query(query, con)
            print(pt.to_string(index=False, col_space=15))
            
            while(1) :
                pt_cost = int(input("원하시는 기간을 정하여 표를 보고 가격을 입력해주세요 : "))
                if (pt_cost in [300000, 500000, 600000, 450000, 700000, 850000, 350000, 460000, 530000]):
                    break
                else :
                    print("없는 가격입니다. 다시 입력해주세요.")
        else :
            pt_cost = 0
            
        Administer_id = int(input("관리자ID (회원에게는 보여주면 안됩니다.) : "))
        
        TotalCost = int(cost) + int(locker_cost) + int(pt_cost)  
        print("\n총 비용은 {}원 입니다.\n회원등록이 완료되었습니다. 감사합니다:)\n".format(TotalCost))
        print("------------------------------------------------")
            
        data_C = [(Member_id, Name, Birthday, Age_group, Address, Phone,  Gender, Locker_useage, Trainer_id, Administer_id, TotalCost)]
        
        cursor_db = con.cursor()
        cursor_db.executemany('''
        INSERT INTO Member 
        (Member_id, Name, Birthday, Age_group, Address, Phone,  Gender, Locker_useage, Trainer_id, Administer_id, TotalCost)
        VALUES(?,?,?,?,?,?,?,?,?,?,?)''', data_C)
        con.commit() 
    
    
    if(num == 2):
        
        print("\n-------------------- 트레이너 모드 ----------------------")
        query = 'SELECT DISTINCT Trainer_id as 트레이너ID, Trainer_name as 이름 FROM Trainer'
        traniner = pd.read_sql_query(query, con)
        print(traniner.to_string(index=False,col_space=13))
        Trainer = input("트레이너ID를 입력 : ")
        
        print("\n--------------------- 지정회원 목록 --------------------- ")
        query = f'SELECT DISTINCT Name, Locker_id, Gender, Phone FROM Member Where Trainer_id = "{Trainer}"'
        t = pd.read_sql_query(query, con)
        print(t.to_string(index=False))
        print("--------------------------------------------------------")
        
        
    if(num == 3):
        
        print("\n--------------------- 관리자모드 -----------------------\n ** 관리자ID는 본인만 알고 있습니다. **\n")
        query = 'SELECT DISTINCT Administer_name as 관리자목록 FROM Administer'
        adm = pd.read_sql_query(query, con)
        print(adm.to_string(index=False))
        print("--------------------------------------------------------")
        
        adm_id = int(input("\n관리자ID를 입력해주세요 : "))
        if (adm_id == 1001) :
            print("이강욱님, 안녕하세요")
        elif (adm_id == 1002) :
            print("정현경님, 안녕하세요")
        else :
            print("존재하지 않는 관리자ID 입니다.")
            break
        
        num2 = 0
        
        while(num2 != 5): 
            print("\n\n--------------------------------------------------------")
            print("1. 기구관리 \n2. 라커관리\n3. 트레이너관리\n4. 회원목록\n5. 종료")
            num2 = int(input("번호 선택 : "))
            print("--------------------------------------------------------")
        
            if(num2 == 1):
                print("")
                query = 'SELECT DISTINCT Equipment_id as 기구이름, Equipment_Check as 상태 FROM Equipment'
                eq = pd.read_sql_query(query, con)
                print(eq)
            
            if(num2 == 2):
                print("")
                print("< 라커 목록 >")
                query = 'SELECT DISTINCT Locker_id , Locker_gender as 성별 FROM Locker'
                locker = pd.read_sql_query(query, con)
                print(locker.to_string(index=False))
                
                print("")
                print("< 사용중인 라커 목록 >")
                query = 'SELECT DISTINCT Locker.Locker_id , Locker_gender as 성별, Name as 이름 FROM Locker, Member WHERE Locker.Locker_id = Member.Locker_id'
                locker = pd.read_sql_query(query, con)
                print(locker.to_string(index=False))
                
                print("")
                print("< 회원 목록 >")
                query = 'SELECT DISTINCT Name as 이름, Locker_useage as 라커사용_유무, Locker_id  FROM Member'
                member = pd.read_sql_query(query, con)
                print(member.to_string(index=False))
            
                locker_num = input("\n라커 번호를 부여 (yes/no) :")
                if (locker_num == "yes" or locker_num == "네") :
                    cursor_db = con.cursor()
                    name = input("회원이름 : ")
                    locker_id = input("부여할 라커ID : ")
                    query = 'UPDATE Member SET Locker_id=? where Name=?'
                    cursor_db.execute(query, (locker_id, name))
                    con.commit() 
            
            if(num2 == 3):
                print("")
                query = 'SELECT DISTINCT Member_id, Name as 회원이름, t.Trainer_id as 트레이너ID, Trainer_name as 트레이너이름 FROM Trainer t, Member m WHERE t.Trainer_id = m.Trainer_id'
                traniner = pd.read_sql_query(query, con)
                print(traniner.to_string(index=False))      
                
            if(num2 == 4):
                print("")
                query = 'SELECT DISTINCT * FROM Member'
                member = pd.read_sql_query(query, con)
                print(member.to_string(index=False, col_space=15))
                
                member_list = []

                for index, row in member.iterrows():
                    member_dict = {}
                    member_dict['Memb_id'] = row['Member_id']
                    member_dict['Name'] = row['Name']
                    member_dict['Birthday'] = row['Birthday']
                    member_dict['Address'] = row['Address']
                    member_dict['Phone'] = row['Phone']
                    member_dict['Gender'] = row['Gender']
    
                    member_list.append(member_dict)


query = "SELECT * FROM Member;"
df_member = pd.read_sql_query(query, conn)

with pd.ExcelWriter(output_file) as writer:
    df_member.to_excel(writer, sheet_name='Member', index=False)
    
con.close()

Excel 파일이 생성되었습니다.



**************** 헬스장 관리 데이터베이스 *****************
*    1. 회원등록
*    2. 트레이너 모드
*    3. 관리자 모드
*    4. 종료
*********************************************************


번호를 선택하세요 :  1



------------ 회원등록을 시작하겠습니다 ------------


회원ID :  A234
이름 :  정현경
생년월일 :  020515
연령층 :  성인
거주지 :  진주
휴대폰 번호 :  01013456789
성별(남/여) :  여



< 등록금 >
      Age_group        OneMonth      ThreeMonth        SixMonth            Year
     청소년(14~19)           50000          120000          190000          330000
      성인(20~59)           80000          150000          250000          450000
        노년(60~)           70000          130000          210000          370000


...



--------------------------------------------------------
1. 기구관리 
2. 라커관리
3. 트레이너관리
4. 회원목록
5. 종료


번호 선택 :  3


--------------------------------------------------------

Member_id 회원이름 트레이너ID 트레이너이름
     A123  정현경    J06    주민수
     Q012  이강욱    R01   로니콜먼


--------------------------------------------------------
1. 기구관리 
2. 라커관리
3. 트레이너관리
4. 회원목록
5. 종료


번호 선택 :  4


--------------------------------------------------------

      Member_id            Name        Birthday       Age_group         Address           Phone          Gender   Locker_useage       Locker_id      Trainer_id   Administer_id       TotalCost
           Q012             이강욱          000428              성인              김해     01033944618               남               네               1             R01            1001          610000
           A123             정현경          020515              성인              진주     01012345678               여               네              11             J06            1001          540000


--------------------------------------------------------
1. 기구관리 
2. 라커관리
3. 트레이너관리
4. 회원목록
5. 종료
